In [1]:
from src.auth.aurora import MySQL

In [2]:
mySQL = MySQL()

In [3]:
query = """SELECT id, name, email
FROM api_users
WHERE id IN (
	SELECT DISTINCT user_id
    FROM topaz.api_service_tests
    WHERE id >= 12515000
)"""
users = mySQL.execute_query_dict(
    query=query,
)

In [4]:
from src.utils.auth_trees import get_user_clients

all_users = []

for user in users:
    user_id = user["id"]
    core_clients = get_user_clients(
        user_id=user_id,
        mySQL=mySQL,
    )
    user["businesses"] = set()
    for client_id in core_clients:
        query = "SELECT parent_id, id, name FROM api_clients WHERE id = %(client_id)s"
        client_info = mySQL.execute_query_dict(
            query,
            params={
                "client_id": client_id,
            }
        )[0]
        if client_info["id"] == 1:
            continue
        while client_info["parent_id"] not in (1, 23):
            client_id = client_info["parent_id"]
            client_info = mySQL.execute_query_dict(
                query,
                params={
                    "client_id": client_id,
                }
            )[0]
        if client_info["parent_id"] in (1, 23):
            user["businesses"] |= {client_info["name"]}
    if len(user["businesses"]) > 0:
        all_users.append(user)
        print(user)

{'id': 117, 'name': 'Andrew Tapping', 'email': 'andrew@armadilloshield.co.uk', 'businesses': {'National Trust', 'Birmingham Diocesan Trust', 'SS Systems'}}
{'id': 143, 'name': 'Jack Peach', 'email': 'jack.peach@triman.co.uk', 'businesses': {'Tri Management'}}
{'id': 288, 'name': 'Alan Wells', 'email': 'alan.wells@triman.co.uk', 'businesses': {'CSS Systems Ltd', 'Tri Management'}}
{'id': 351, 'name': 'Gary Firth', 'email': 'gary.firth@rosse-systems.co.uk', 'businesses': {'Rosse Systems', 'Rosse Systems (NT)'}}
{'id': 382, 'name': 'Dave Hutton', 'email': 'dave.hutton@triman.co.uk', 'businesses': {'Tri Management'}}
{'id': 443, 'name': 'Tony Jenkins', 'email': 'tony@valefireandsecurity.uk', 'businesses': {'Vale Fire & Security Ltd', 'ISS'}}
{'id': 587, 'name': 'Tom Doran (Primary)', 'email': 't.doran@masterfire.ie', 'businesses': {'MasterFire'}}
{'id': 613, 'name': 'Gavin Smallwood', 'email': 'gavin.smallwood@sunfishservices.com', 'businesses': {'Sunfish Services', 'Sunfish Services (NT)'

In [5]:
import pandas as pd
df = pd.DataFrame(all_users)

In [ ]:
df.to_csv("data/2025-05-15 Engineers.csv")